In [4]:
!pip install pandas openpyxl networkx

In [5]:
# @title Импорты
from dataclasses import dataclass
from datetime import datetime
import pandas as pd
import os
import numpy as np
import networkx as nx
from collections import Counter, defaultdict
import math
from datetime import datetime, timedelta
from typing import List, Dict, Set, Tuple, Optional
from tqdm import tqdm
import copy

In [6]:
# @title Модель заявки
def safe_convert_to_float(value):
    """
    Безопасно конвертирует значение в float, обрабатывая nan и None

    Args:
        value: Значение для конвертации

    Returns:
        Optional[float]: Конвертированное значение или None
    """
    if value is None or pd.isna(value) or (isinstance(value, float) and math.isnan(value)):
        return None
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def safe_convert_to_str(value):
    """
    Безопасно конвертирует значение в строку, обрабатывая nan и None

    Args:
        value: Значение для конвертации

    Returns:
        str: Конвертированное значение или пустая строка
    """
    if value is None or pd.isna(value) or (isinstance(value, float) and math.isnan(value)):
        return ""
    return str(value)

@dataclass
class Location:
    """
    Класс, представляющий местоположение

    Attributes:
        latitude: Широта
        longitude: Долгота
        city: Название города
        region: Название региона
    """
    latitude: Optional[float]
    longitude: Optional[float]
    city: str
    region: str
    cluster: Optional[str]

@dataclass
class Order:
    """
    Класс, представляющий заявку на перевозку

    Attributes:
        order_number: Номер заявки
        published_at: Дата и время публикации заявки
        first_point_date_time_start: Дата и время начала работы в первой точке
        first_point_location: Местоположение первой точки
        last_point_date_time_until: Дата и время окончания работы в последней точке
        last_point_location: Местоположение последней точки
        customer_price_with_vat: Цена заказа для клиента
        carrier_price_with_vat: Цена заказа для перевозчика
        auction_price: Цена заказа из аукциона
    """
    order_number: str
    published_at: Optional[datetime]
    customer_name: Optional[str]
    manager_name: Optional[str]

    # Первая точка
    first_point_date_time_start: Optional[datetime]
    first_point_location: Location

    # Последняя точка
    last_point_date_time_until: Optional[datetime]
    last_point_location: Location

    # Цена заказа
    customer_price_with_vat: Optional[float]
    carrier_price_with_vat: Optional[float]
    auction_price: Optional[float]

    # Расстояние между точками в км
    distance: Optional[float]

    @classmethod
    def from_row(cls, row):
        """
        Создает объект Order из строки данных

        Args:
            row: Строка данных из DataFrame

        Returns:
            Order: Объект заявки
        """
        first_point_location = Location(
            latitude=safe_convert_to_float(row['first_point_location_lat']),
            longitude=safe_convert_to_float(row['first_point_location_lon']),
            city=safe_convert_to_str(row['first_point_location_city']),
            region=safe_convert_to_str(row['first_point_location_region']),
            cluster=None
        )

        last_point_location = Location(
            latitude=safe_convert_to_float(row['last_point_location_lat']),
            longitude=safe_convert_to_float(row['last_point_location_lon']),
            city=safe_convert_to_str(row['last_point_location_city']),
            region=safe_convert_to_str(row['last_point_location_region']),
            cluster=None
        )

        return cls(
            order_number=str(row['order_number']),
            published_at= row['published_at'] if type(row['published_at']) != str else datetime.strptime(row['published_at'].split('.')[0], '%Y-%m-%d %H:%M:%S'),
            customer_name=row['customer_name'],
            manager_name=row['manager_name'],
            first_point_date_time_start=row['first_point_date_time_start'] if type(row['first_point_date_time_start']) != str else datetime.strptime(row['first_point_date_time_start'].split('.')[0], '%Y-%m-%d %H:%M:%S'),
            first_point_location=first_point_location,
            last_point_date_time_until=row['last_point_date_time_until'] if type(row['last_point_date_time_until']) != str else datetime.strptime(row['last_point_date_time_until'].split('.')[0], '%Y-%m-%d %H:%M:%S'),
            last_point_location=last_point_location,
            customer_price_with_vat=safe_convert_to_float(row['customer_price_with_vat']) or 0,
            carrier_price_with_vat=safe_convert_to_float(row['carrier_price_with_vat']) or 0,
            auction_price=safe_convert_to_float(row['auction_price']) or 0,
            distance=safe_convert_to_float(row['distance_km']) or 0
        )

    @classmethod
    def synthetic(cls, date_time_start: datetime, date_time_end: datetime):
        return cls(
            order_number='synthetic',
            published_at=date_time_start,
            customer_name='',
            manager_name='',
            first_point_date_time_start=date_time_start,
            first_point_location=Location(latitude=None, longitude=None, city='', region='', cluster=None),
            last_point_date_time_until=date_time_end,
            last_point_location=Location(latitude=None, longitude=None, city='', region='', cluster=None),
            customer_price_with_vat=0,
            carrier_price_with_vat=0,
            auction_price=0,
            distance=0
        )

    def get_weekday(self) -> int:
        """
        Возвращает день недели даты публикации заявки

        Returns:
            int: День недели (0 - понедельник, 6 - воскресенье)
        """
        return self.published_at.weekday()

    def get_weekday_name(self) -> str:
        """
        Возвращает название дня недели даты публикации заявки на русском языке

        Returns:
            str: Название дня недели (Понедельник, Вторник, и т.д.)
        """
        return self.published_at.strftime('%A').capitalize()

    def get_travel_time(self) -> Optional[float]:
        """
        Рассчитывает время в пути в часах, без учета разгрузки и стоянки

        Returns:
            Optional[float]: Время в пути в часах или None, если данные недоступны
        """
        if self.get_start_datetime() and self.get_end_datetime():
            return (self.get_end_datetime() - self.get_start_datetime()).total_seconds() / 3600
        return None

    def get_travel_time_group(self) -> Optional[int]:
        """
        Возвращает группу времени в пути, без учета разгрузки и стоянки
        """
        if self.get_travel_time():
            return int(((self.get_travel_time() - 0.00001) // 48) + 1)
        return None

    def get_price(self) -> float:
        """
        Возвращает цену заказа

        Returns:
            float: Цена заказа
        """
        #return self.customer_price_with_vat
        return self.auction_price

    def get_start_datetime(self) -> datetime:
        """
        Возвращает дату и время начала работы в первой точке

        Returns:
            datetime: Дата и время начала работы в первой точке
        """
        return self.first_point_date_time_start

    def get_end_datetime(self) -> datetime:
        """
        Возвращает дату и время окончания работы в последней точке

        Returns:
            datetime: Дата и время окончания работы в последней точке
        """
        return self.last_point_date_time_until

    def set_coordinates_from(self, latitude: float, longitude: float):
        self.first_point_location.latitude = latitude
        self.first_point_location.longitude = longitude

    def set_coordinates_to(self, latitude: float, longitude: float):
        self.last_point_location.latitude = latitude
        self.last_point_location.longitude = longitude

    def set_clusters(self, from_cluster: str, to_cluster: str):
        self.first_point_location.cluster = from_cluster
        self.last_point_location.cluster = to_cluster

    def get_cluster_from(self) -> str:
        return self.first_point_location.cluster

    def get_cluster_to(self) -> str:
        return self.last_point_location.cluster

    def get_coordinates_from(self) -> np.ndarray:
        return np.array([self.first_point_location.latitude, self.first_point_location.longitude])

    def get_coordinates_to(self) -> np.ndarray:
        return np.array([self.last_point_location.latitude, self.last_point_location.longitude])

    def has_coordinates(self) -> bool:
        """
        Проверяет, есть ли у заявки координаты для отправления и назначения

        Returns:
            bool: True если есть координаты для обеих точек, False иначе
        """
        return (self.first_point_location.latitude is not None and
                self.first_point_location.longitude is not None and
                self.last_point_location.latitude is not None and
                self.last_point_location.longitude is not None)

    def get_customer_name(self) -> str:
        return self.customer_name

    def get_manager_name(self) -> str:
        return self.manager_name

    def get_km_per_day(self, unloading_hours: float = 6) -> float:
        """
        Возвращает плановый пробег за сутки
        """
        travel_time_days = (self.get_travel_time() - unloading_hours) / 24
        return self.distance / travel_time_days if travel_time_days > 0 else 0

In [7]:
# @title Методы считывания заявок
def calculate_average_coordinates(orders: List[Order]) -> Dict[str, Tuple[float, float]]:
    """
    Вычисляет средние координаты для каждого города на основе заявок с координатами

    Args:
        orders (List[Order]): Список заявок

    Returns:
        Dict[str, Tuple[float, float]]: Словарь с городом в качестве ключа и кортежем (широта, долгота) в качестве значения
    """
    city_coordinates = defaultdict(list)

    # Собираем все координаты для каждого города
    for order in orders:
        # Координаты города отправления
        if (order.first_point_location.latitude is not None and
            order.first_point_location.longitude is not None and
            order.first_point_location.city):
            city_coordinates[order.first_point_location.city].append(
                (order.first_point_location.latitude, order.first_point_location.longitude)
            )

        # Координаты города назначения
        if (order.last_point_location.latitude is not None and
            order.last_point_location.longitude is not None and
            order.last_point_location.city):
            city_coordinates[order.last_point_location.city].append(
                (order.last_point_location.latitude, order.last_point_location.longitude)
            )

    # Вычисляем средние координаты для каждого города
    average_coordinates = {}
    for city, coordinates in city_coordinates.items():
        if coordinates:
            avg_lat = sum(coord[0] for coord in coordinates) / len(coordinates)
            avg_lon = sum(coord[1] for coord in coordinates) / len(coordinates)
            average_coordinates[city] = (avg_lat, avg_lon)

    return average_coordinates

def fill_missing_coordinates(orders: List[Order]) -> List[Order]:
    """
    Заполняет отсутствующие координаты средними значениями для соответствующих городов

    Args:
        orders (List[Order]): Список заявок

    Returns:
        List[Order]: Список заявок с заполненными координатами
    """
    # Вычисляем средние координаты для всех городов
    average_coordinates = calculate_average_coordinates(orders)

    for order in orders:
        first_location = order.first_point_location
        last_location = order.last_point_location

        # Заполняем координаты для города отправления, если они отсутствуют
        if (first_location.latitude is None or first_location.longitude is None) and first_location.city:
            if first_location.city in average_coordinates:
                avg_lat, avg_lon = average_coordinates[first_location.city]
                order.set_coordinates_from(avg_lat, avg_lon)

        # Заполняем координаты для города назначения, если они отсутствуют
        if (last_location.latitude is None or last_location.longitude is None) and last_location.city:
            if last_location.city in average_coordinates:
                avg_lat, avg_lon = average_coordinates[last_location.city]
                order.set_coordinates_to(avg_lat, avg_lon)

    return orders

def get_orders_from_csv(filename: str) -> List[Order]:
    """
    Читает данные о заявках из csv файла

    Args:
        filename (str): Имя файла csv
    """
    df = pd.read_csv(filename)
    orders = [Order.from_row(row) for _, row in df.iterrows()]
    print(f"\nУспешно создано объектов Order: {len(orders)}")
    return orders

def get_orders(verbose_excel: bool = False, unloading_hours: float = 6, max_km_per_day: float = 800) -> List[Order]:
    """
    Получает данные о заявках и создает коллекцию объектов Order

    Args:
        verbose_excel: Флаг для вывода детализации в файл

    Returns:
        List[Order]: Список объектов Order
    """
    # Создаем коллекцию объектов Order
    orders = get_orders_from_csv("orders_source.csv")
    print(f"Доля заявок с координатами: {len(get_orders_with_coordinates(orders))*1.0/len(orders)}")
    # Заполняем отсутствующие координаты средними значениями
    orders = fill_missing_coordinates(orders)
    print(f"Доля заявок с координатами после заполнения средними значениями: {len(get_orders_with_coordinates(orders))*1.0/len(orders)}")

    # Удаляем заявки без координат
    orders = get_orders_with_coordinates(orders)
    print(f"Итоговое количество заявок до фильтрации: {len(orders)}")

    # Удаляем заявки с клиентами из дополнительного списка, по которым есть ограничения в работе
    orders = filter_orders_by_customers(orders)
    if verbose_excel:
        orders_to_xlsx(orders, "orders_before_km_per_day_filter.xlsx", unloading_hours=unloading_hours)

    # Убираем заявки с аномально большим пробегом за сутки
    print_km_per_day_percentiles(orders, unloading_hours)
    orders = [order for order in orders if order.get_km_per_day(unloading_hours) <= max_km_per_day and order.get_km_per_day(unloading_hours) > 0]
    print(f"Оставлено заявок с пробегом за сутки <= {max_km_per_day} км: {len(orders)}")
    if verbose_excel:
        orders_to_xlsx(orders, "orders.xlsx", unloading_hours=unloading_hours)

    return orders

def filter_orders_by_customers(orders: list) -> list:
    """
    Фильтрует заявки, исключая клиентов, у которых есть ограничения в работе (не сотрудничаем), согласно Google Sheets по заданной ссылке.
    Сравнение наименований клиентов производится в нижнем регистре.
    """

    # Ссылка на Google Sheets (формируем ссылку для экспорта в CSV)
    sheet_url = "https://docs.google.com/spreadsheets/d/1UM7vi2ijVP8DMskEa2QHp8Us937Cwj6fFria4vzvRj0/export?format=csv&gid=0"
    try:
        df = pd.read_csv(sheet_url)
    except Exception as e:
        print(f"Не удалось загрузить Google Sheets: {e}")
        return orders

    # Проверяем наличие нужных столбцов
    if "Наименование" not in df.columns or "Ограничения в работе (не сотрудничаем)" not in df.columns:
        print("В Google Sheets отсутствуют необходимые столбцы 'Наименование' или 'Ограничения в работе (не сотрудничаем)'")
        return orders

    # Получаем список клиентов с ограничениями (в нижнем регистре)
    restricted_customers = set(
        df.loc[
            df["Ограничения в работе (не сотрудничаем)"].notnull() &
            (df["Ограничения в работе (не сотрудничаем)"].astype(str).str.strip() != ""),
            "Наименование"
        ].astype(str).str.strip().str.lower()
    )

    filtered_orders = [
        order for order in orders
        if order.get_customer_name().strip().lower() not in restricted_customers
    ]

    print(f"Исключено заявок по клиентам с ограничениями из дополнительного списка: {len(orders) - len(filtered_orders)}")
    return filtered_orders

def get_orders_with_coordinates(orders: List[Order]) -> List[Order]:
    """
    Получает заявки без координат
    """
    return [order for order in orders
            if (order.first_point_location.latitude is not None and order.first_point_location.longitude is not None and
                order.last_point_location.latitude is not None and order.last_point_location.longitude is not None)]

def orders_to_xlsx(orders: List[Order], filename: str, unloading_hours: float = 6):
    """
    Сохраняет список заявок в Excel файл с детальной информацией

    Args:
        orders (List[Order]): Список заявок
        filename (str): Имя файла Excel
    """
    # Создаем DataFrame из списка заявок
    data = []
    for order in orders:
        data.append({
            'Номер заявки': order.order_number,
            'Дата публикации': order.published_at,
            'Менеджер': order.get_manager_name(),
            'Клиент': order.get_customer_name(),
            'Город отправления': order.first_point_location.city,
            'Регион отправления': order.first_point_location.region,
            'Дата отправления': order.get_start_datetime().strftime('%Y-%m-%d %H:%M:%S'),
            'Город назначения': order.last_point_location.city,
            'Регион назначения': order.last_point_location.region,
            'Дата прибытия': order.get_end_datetime().strftime('%Y-%m-%d %H:%M:%S'),
            'Время в пути, ч.': order.get_travel_time(),
            'Время в пути, группа по 48 ч.': order.get_travel_time_group(),
            'Цена заказа, руб.': order.get_price(),
            'Расстояние, км': order.distance,
            'Пробег за сутки, км': order.get_km_per_day(unloading_hours)
        })

    df = pd.DataFrame(data)

    # Сохраняем в Excel
    output_path = os.path.join("output", filename)
    os.makedirs("output", exist_ok=True)

    df.to_excel(output_path, index=False, sheet_name='Orders')
    print(f"Данные успешно сохранены в файл: {output_path}")

def filter_orders_by_cluster_price_median(orders: list) -> list:
    """
    Убирает заявки, у которых цена отклоняется более чем на 50% по модулю от медианы по связке Кластер-Кластер.

    Args:
        orders (list): Список заявок (Order)

    Returns:
        list: Отфильтрованный список заявок
    """
    # Формируем DataFrame из заявок
    data = []
    for order in orders:
        data.append({
            'order': order,
            'from_cluster': order.get_cluster_from(),
            'to_cluster': order.get_cluster_to(),
            'price': order.get_price()
        })
    df = pd.DataFrame(data)

    # Считаем медиану по каждой связке кластер-откуда -> кластер-куда
    df['median'] = (
        df.groupby(['from_cluster', 'to_cluster'])['price']
        .transform('median')
    )

    # Вычисляем отклонение от медианы
    df['deviation'] = abs(df['price'] / df['median'] - 1)

    # Фильтруем заявки: только те, у которых отклонение <= 0.5
    filtered_orders = df[df['deviation'] <= 0.5]['order'].tolist()
    return filtered_orders

def print_km_per_day_percentiles(orders: list, unloading_hours: float = 6):
    """
    Выводит в консоль распределение пробега за сутки по персентилям.

    Args:
        orders (list): Список заявок (Order)
        unloading_hours (float): Нормативная длительность ПРР в часах (по умолчанию 6)
    """

    km_per_day_values = [
        order.get_km_per_day(unloading_hours)
        for order in orders
    ]

    percentiles = [0, 5, 10, 25, 50, 75, 90, 95, 99, 100]
    values = np.percentile(km_per_day_values, percentiles)

    print("Распределение пробега за сутки по персентилям (км/сутки):")
    for p, v in zip(percentiles, values):
        print(f"{p:>3} персентиль: {v:.1f}")

In [8]:
# Получаем данные о заявках
unloading_hours = 6 # нормативная длительность ПРР в часах
max_km_per_day = 850 # максимальный пробег за сутки в км
orders = get_orders(verbose_excel=False, unloading_hours=unloading_hours, max_km_per_day=max_km_per_day)


Успешно создано объектов Order: 115104
Доля заявок с координатами: 0.48536975257158743
Доля заявок с координатами после заполнения средними значениями: 0.9605226577703642
Итоговое количество заявок до фильтрации: 110560
Исключено заявок по клиентам с ограничениями из дополнительного списка: 4068
Распределение пробега за сутки по персентилям (км/сутки):
  0 персентиль: 0.0
  5 персентиль: 0.0
 10 персентиль: 65.8
 25 персентиль: 412.9
 50 персентиль: 596.4
 75 персентиль: 955.3
 90 персентиль: 1571.0
 95 персентиль: 2186.1
 99 персентиль: 5996.7
100 персентиль: 459399.8
Оставлено заявок с пробегом за сутки <= 850 км: 65554


In [9]:
# Сохраняем в excel
orders_to_xlsx(orders, "orders.xlsx", unloading_hours)

Данные успешно сохранены в файл: output\orders.xlsx


In [11]:
# @title Методы для разделения выборки
def split_orders(orders: List[Order], verbose_excel: bool = False) -> Tuple[List[Order], List[Order]]:
    """
    Разделяет заявки на обучающую и контрольную выборки

    Args:
        orders (List[Order]): Список заявок
        verbose_excel: Флаг для вывода детализации в файл
    Returns:
        Tuple[List[Order], List[Order]]: Кортеж из двух списков (обучающая выборка, контрольная выборка)
    """

    # Разделяем на обучающую и контрольную выборки
    train_orders = []
    control_orders = []
    threshold_date = max(order.get_start_datetime().date() for order in orders) - timedelta(days=30)

    for order in orders:
        start_datetime = order.get_start_datetime()
        # Преобразуем дату в naive формат, если она aware
        if start_datetime.tzinfo is not None:
            start_datetime = start_datetime.replace(tzinfo=None)

        if start_datetime.date() < threshold_date:
            train_orders.append(order)
        else:
            control_orders.append(order)

    print(f"\nРазделение заявок:")
    print(f"Обучающая выборка: {len(train_orders)} заявок")
    print(f"Контрольная выборка: {len(control_orders)} заявок")

    # Сохраняем разделенные выборки в Excel
    if verbose_excel:
        splitted_orders_to_xlsx(train_orders, control_orders, "splitted_orders.xlsx")

    return train_orders, control_orders

def splitted_orders_to_xlsx(train_orders: List[Order], control_orders: List[Order], filename: str):
    """
    Сохраняет разделенные выборки заявок в Excel файл

    Args:
        train_orders (List[Order]): Список заявок для обучения
        control_orders (List[Order]): Список заявок для контроля
        filename (str): Имя файла Excel
    """
    # Создаем DataFrame из списка заявок
    data = []

    # Добавляем заявки для обучения
    for order in train_orders:
        data.append({
            'Номер заявки': order.order_number,
            'Дата публикации': order.published_at,
            'Город отправления': order.first_point_location.city,
            'Регион отправления': order.first_point_location.region,
            'Дата отправления': order.get_start_datetime().strftime('%Y-%m-%d %H:%M:%S'),
            'Город назначения': order.last_point_location.city,
            'Регион назначения': order.last_point_location.region,
            'Дата прибытия': order.get_end_datetime().strftime('%Y-%m-%d %H:%M:%S'),
            'Время в пути, ч.': order.get_travel_time(),
            'Цена заказа, руб.': order.get_price(),
            'Выборка': 'Обучение'
        })

    # Добавляем заявки для контроля
    for order in control_orders:
        data.append({
            'Номер заявки': order.order_number,
            'Дата публикации': order.published_at,
            'Город отправления': order.first_point_location.city,
            'Регион отправления': order.first_point_location.region,
            'Дата отправления': order.get_start_datetime().strftime('%Y-%m-%d %H:%M:%S'),
            'Город назначения': order.last_point_location.city,
            'Регион назначения': order.last_point_location.region,
            'Дата прибытия': order.get_end_datetime().strftime('%Y-%m-%d %H:%M:%S'),
            'Время в пути, ч.': order.get_travel_time(),
            'Цена заказа, руб.': order.get_price(),
            'Выборка': 'Контроль'
        })

    df = pd.DataFrame(data)

    # Сохраняем в Excel
    output_path = os.path.join("output", filename)
    os.makedirs("output", exist_ok=True)

    df.to_excel(output_path, index=False, sheet_name='SplittedOrders')
    print(f"Данные успешно сохранены в файл: {output_path}")

In [12]:
# Разделяем заявки на обучающую и контрольную выборки
train_orders, control_orders = split_orders(orders)


Разделение заявок:
Обучающая выборка: 31133 заявок
Контрольная выборка: 34421 заявок


In [13]:
# Сохраняем в excel
splitted_orders_to_xlsx(train_orders, control_orders, "splitted_orders.xlsx")

Данные успешно сохранены в файл: output\splitted_orders.xlsx


In [14]:
# @title Методы кластеризации по координатам
def haversine_distance(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    """
    Вычисляет расстояние между двумя точками по формуле гаверсинуса (в километрах)

    Args:
        lat1, lon1: Координаты первой точки
        lat2, lon2: Координаты второй точки

    Returns:
        float: Расстояние в километрах
    """
    # Конвертируем градусы в радианы
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Разность координат
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Формула гаверсинуса
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.asin(math.sqrt(a))

    # Радиус Земли в километрах
    r = 6371

    return c * r

def extract_coordinates_from_orders(orders: List[Order]) -> Tuple[np.ndarray, List[str]]:
    """
    Извлекает уникальные координаты из заявок для кластеризации вместе с названиями населенных пунктов

    Args:
        orders: Список заявок

    Returns:
        Tuple[np.ndarray, List[str]]: Уникальные координаты и соответствующие названия населенных пунктов
    """
    coordinates = []
    city_names = []

    for order in orders:
        # Проверяем, что у заявки есть координаты
        if order.has_coordinates:
            # Добавляем координаты отправления
            coordinates.append([
                order.first_point_location.latitude,
                order.first_point_location.longitude
            ])
            city_names.append(order.first_point_location.city)

            # Добавляем координаты назначения
            coordinates.append([
                order.last_point_location.latitude,
                order.last_point_location.longitude
            ])
            city_names.append(order.last_point_location.city)

    # Удаляем дубликаты, сохраняя наиболее часто встречающиеся названия городов
    unique_coordinates = []
    unique_cities = []
    coords_to_cities = {}  # Словарь для группировки городов по координатам

    for i in range(len(coordinates)):
        coords = coordinates[i]
        coords_tuple = tuple(coords)
        if coords_tuple not in coords_to_cities:
            coords_to_cities[coords_tuple] = []
        coords_to_cities[coords_tuple].append(city_names[i])

    # Для каждой уникальной координаты выбираем наиболее часто встречающийся город
    for coords_tuple, cities in coords_to_cities.items():
        unique_coordinates.append(list(coords_tuple))
        # Подсчитываем частоту каждого города
        city_counter = Counter(cities)
        # Берем самый частый город
        most_common_city = city_counter.most_common(1)[0][0]
        unique_cities.append(most_common_city)

    return np.array(unique_coordinates), unique_cities

def cluster_coordinates(coordinates: np.ndarray,
                      city_names: List[str],
                      radius: float,
                      verbose: bool = False) -> Dict[str, np.ndarray]:
    """
    Кластеризует координаты с гарантией, что все точки в кластере находятся в пределах радиуса от центроида

    Args:
        coordinates: Массив координат [lat, lon]
        city_names: Список названий городов, соответствующих координатам
        radius: Радиус кластера - максимальное расстояние от центроида до точек в кластере (в км)
        verbose: Флаг для вывода детализации в консоль
    Returns:
        Dict[str, np.ndarray]: Словарь с названиями кластеров и их центроидами
    """
    n_points = len(coordinates)
    cluster_labels = np.full(n_points, -1)  # -1 означает "не кластеризовано"
    cluster_centroids = {}
    cluster_cities = {}  # Словарь для хранения городов в каждом кластере
    current_cluster_id = 0

    # Создаем маску для некластеризованных точек
    unclustered_mask = np.ones(n_points, dtype=bool)

    while np.any(unclustered_mask):
        # Выбираем случайную некластеризованную точку как начальный центроид
        unclustered_indices = np.where(unclustered_mask)[0]

        start_idx = unclustered_indices[0]
        centroid = coordinates[start_idx].copy()

        # Находим все точки в пределах радиуса от центроида
        cluster_points = []
        cluster_indices = []
        cluster_point_cities = []

        for i in range(n_points):
            if unclustered_mask[i]:
                dist = haversine_distance(coordinates[i][0], coordinates[i][1],
                                        centroid[0], centroid[1])
                if dist < radius:
                    cluster_points.append(coordinates[i])
                    cluster_indices.append(i)
                    cluster_point_cities.append(city_names[i])

        # Обновляем центроид как среднее всех точек в кластере
        cluster_points_array = np.array(cluster_points)
        centroid = np.mean(cluster_points_array, axis=0)

        # Проверяем, что все точки действительно в пределах радиуса от нового центроида
        valid_points = []
        valid_indices = []
        valid_cities = []

        for i, point in enumerate(cluster_points):
            dist = haversine_distance(point[0], point[1], centroid[0], centroid[1])
            if dist <= radius:
                valid_points.append(point)
                valid_indices.append(cluster_indices[i])
                valid_cities.append(cluster_point_cities[i])

        if len(valid_points) >= 1:  # Минимум одна точка (начальная точка)
            # Сохраняем города для этого кластера
            cluster_cities[current_cluster_id] = valid_cities

            # Помечаем точки как кластеризованные
            for idx in valid_indices:
                cluster_labels[idx] = current_cluster_id
                unclustered_mask[idx] = False

            # Выводим информацию о кластере
            max_distance = 0
            for point in valid_points:
                dist = haversine_distance(point[0], point[1], centroid[0], centroid[1])
                max_distance = max(max_distance, dist)
            if verbose:
                print(f"Кластер {current_cluster_id}: {len(valid_points)} точек с максимальным расстоянием {max_distance:.1f} км")
            current_cluster_id += 1
        else:
            # Помечаем начальную точку как шум
            unclustered_mask[start_idx] = False

    # Определяем названия кластеров на основе наиболее часто встречающихся городов
    # и создаем cluster_centroids с названиями вместо ID
    for cluster_id, cities in cluster_cities.items():
        if cities:
            # Подсчитываем частоту каждого города
            city_counter = Counter(cities)
            # Берем самый частый город
            most_common_city = city_counter.most_common(1)[0][0]
            most_common_city = f"{most_common_city} (кластер {cluster_id})"

            # Находим центроид для этого кластера
            cluster_points = []
            for i in range(n_points):
                if cluster_labels[i] == cluster_id:
                    cluster_points.append(coordinates[i])

            if cluster_points:
                cluster_centroids[most_common_city] = np.mean(cluster_points, axis=0)

    # Выводим общую статистику
    print(f"Всего кластеров: {len(cluster_centroids)}")

    return cluster_centroids


def find_cluster_for_point(point_coords: np.ndarray, cluster_centroids: Dict[str, np.ndarray], radius: float) -> str:
    """
    Находит кластер для точки на основе расстояния от центроида кластера

    Args:
        point_coords: Координаты точки [lat, lon]
        cluster_centroids: Словарь с центроидами кластеров (название -> центроид)
        radius: Радиус кластера (в км)
    Returns:
        str: Название кластера ("Не определен" для шумовых точек)
    """
    if not cluster_centroids:
        return "Не определен"

    # Находим ближайший центроид
    min_distance = radius
    closest_cluster = "Не определен"

    for cluster_name, centroid in cluster_centroids.items():
        distance = haversine_distance(point_coords[0], point_coords[1], centroid[0], centroid[1])
        if distance < min_distance:
            min_distance = distance
            closest_cluster = cluster_name

    return closest_cluster


def save_clustering_results(orders: List[Order], cluster_centroids: Dict[str, np.ndarray], radius: float, save_path: str = "output/clusters.xlsx"):
    """
    Сохраняет результаты кластеризации в Excel

    Args:
        orders: Список заявок из обучения
        cluster_centroids: Словарь с центроидами кластеров (название -> центроид)
        radius: Радиус кластера (в км)
        save_path: Путь для сохранения файла
    """
    # Создаем DataFrame с информацией о заявках и их кластерах
    orders_data = []

    for order in orders:
        # Проверяем, что у заявки есть координаты
        if order.has_coordinates:
            # Координаты отправления
            from_coords = np.array([order.first_point_location.latitude, order.first_point_location.longitude])
            from_cluster = find_cluster_for_point(from_coords, cluster_centroids, radius)

            # Координаты назначения
            to_coords = np.array([order.last_point_location.latitude, order.last_point_location.longitude])
            to_cluster = find_cluster_for_point(to_coords, cluster_centroids, radius)

            # Добавляем информацию о заявке
            orders_data.append({
                'Номер заявки': order.order_number,
                'Город отправления': order.first_point_location.city,
                'Координаты отправления': f"{order.first_point_location.latitude}, {order.first_point_location.longitude}",
                'Город назначения': order.last_point_location.city,
                'Координаты назначения': f"{order.last_point_location.latitude}, {order.last_point_location.longitude}",
                'Кластер города отправления': from_cluster,
                'Кластер города назначения': to_cluster
            })

    df_orders = pd.DataFrame(orders_data)

    # Создаем DataFrame с детализацией по кластерам
    cluster_details = []
    all_coordinates, _ = extract_coordinates_from_orders(orders)
    all_clusters = []
    for coords in all_coordinates:
        all_clusters.append(find_cluster_for_point(coords, cluster_centroids, radius))

    #print(f"\nВосстановлено кластеров: {len(set(all_clusters))}")

    for cluster_name, centroid in cluster_centroids.items():
        # Находим все координаты, принадлежащие этому кластеру
        cluster_coordinates = []
        for i, coords in enumerate(all_coordinates):
            if all_clusters[i] == cluster_name:
                cluster_coordinates.append(coords)

        if cluster_coordinates:
            # Вычисляем статистику
            distances = []
            for coords in cluster_coordinates:
                dist = haversine_distance(coords[0], coords[1], centroid[0], centroid[1])
                distances.append(dist)

            avg_distance = np.mean(distances)
            max_distance = np.max(distances)

            cluster_details.append({
                'Название кластера': cluster_name,
                'Количество координат': len(cluster_coordinates),
                'Центроид (широта)': centroid[0],
                'Центроид (долгота)': centroid[1],
                'Среднее расстояние от центроида (км)': round(avg_distance, 1),
                'Максимальное расстояние от центроида (км)': round(max_distance, 1)
            })

    # Сортируем cluster_details по убыванию количества координат
    cluster_details.sort(key=lambda x: x['Количество координат'], reverse=True)
    df_cluster_details = pd.DataFrame(cluster_details)

    # Сохраняем в Excel
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    with pd.ExcelWriter(save_path, engine='openpyxl') as writer:
        df_orders.to_excel(writer, sheet_name='OrderByCluster', index=False)
        df_cluster_details.to_excel(writer, sheet_name='ClusterDetails', index=False)

    print(f"Результаты кластеризации сохранены в: {save_path}")
    print(f"  - Лист 'OrderByCluster': детализация по заявкам")
    print(f"  - Лист 'ClusterDetails': детализация по кластерам")

def cluster_coordinates_from_train_orders(orders: List[Order], radius: float = 50.0,
                                          verbose_excel: bool = False):
    """
    Основная функция для запуска кластеризации координат

    Args:
        orders: Список заявок
        radius: Радиус кластера (в км)
        verbose_excel: Флаг для вывода детализации в файл
    """
    print(f"\n=== Кластеризация координат отправления и назначения ===")

    # Извлекаем координаты
    print("Извлекаем координаты...")
    coordinates, city_names = extract_coordinates_from_orders(orders)

    if len(coordinates) == 0:
        print("Нет заявок с координатами для кластеризации")
        return

    print(f"Найдено {len(coordinates)} уникальных пар координат")

    # Выполняем кластеризацию с фиксированными параметрами
    print(f"\nВыполняем кластеризацию с параметрами: radius={radius}km")
    cluster_centroids = cluster_coordinates(coordinates, city_names, radius)

    # Сохраняем результаты
    if verbose_excel:
        print("Сохраняем результаты...")
        save_clustering_results(orders, cluster_centroids, radius)

    return cluster_centroids

def aggregate_orders(df: pd.DataFrame, group_cols: list, df_before: pd.DataFrame = None) -> pd.DataFrame:
    """
    Агрегирует заявки по заданным столбцам группировки.

    Args:
        df: DataFrame с заявками (после фильтрации)
        group_cols: Список столбцов для группировки
        df_before: DataFrame с заявками до фильтрации (для медианы до фильтрации)

    Returns:
        DataFrame с агрегированными данными
    """
    # Основная агрегация
    agg = (
        df.groupby(group_cols, as_index=False)
        .agg(
            **{
                'Количество заявок': ('Цена заявки', 'count'),
                'Минимальная цена, руб.': ('Цена заявки', 'min'),
                'Средняя цена, руб.': ('Цена заявки', 'mean'),
                'Медианная цена, руб.': ('Цена заявки', 'median'),
                'Максимальная цена, руб.': ('Цена заявки', 'max'),
            }
        )
    )

    # Медиана и количество заявок до фильтрации
    if df_before is not None:
        agg_before = (
            df_before.groupby(group_cols, as_index=False)
            .agg({
                'Цена заявки': 'median',
                'Номер заявки': 'count' if 'Номер заявки' in df_before.columns else ('Цена заявки', 'count')
            })
        )
        # Переименовываем столбцы
        agg_before = agg_before.rename(columns={
            'Цена заявки': 'Медианная цена до фильтрации, руб.',
            'Номер заявки': 'Количество заявок до фильтрации'
        })
        agg = agg.merge(agg_before, on=group_cols, how='left')
    else:
        agg['Медианная цена до фильтрации, руб.'] = float('nan')
        agg['Количество заявок до фильтрации'] = float('nan')

    # Отклонения
    agg['Отклонение мин. цены от медианы, %'] = 100 * (agg['Минимальная цена, руб.'] / agg['Медианная цена, руб.'] - 1)
    agg['Отклонение макс. цены от медианы, %'] = 100 * (agg['Максимальная цена, руб.'] / agg['Медианная цена, руб.'] - 1)
    agg['Отклонение мин. цены от медианы до фильтрации, %'] = 100 * (agg['Минимальная цена, руб.'] / agg['Медианная цена до фильтрации, руб.'] - 1)
    agg['Отклонение макс. цены от медианы до фильтрации, %'] = 100 * (agg['Максимальная цена, руб.'] / agg['Медианная цена до фильтрации, руб.'] - 1)
    return agg

def orders_with_clusters_to_xlsx(orders: List[Order], orders_before_filtering: List[Order], filename: str = "orders_with_clusters.xlsx"):
    """
    Сохраняет кругорейсы в Excel файл

    Args:
        orders: Список заявок
        orders_before_filtering: Список заявок до фильтрации
        filename (str): Имя файла Excel
    """
    # Формируем DataFrame по заявкам
    data = []
    for order in tqdm(orders, desc="Сохранение заявок", unit="заявка"):
        data.append({
            'Номер заявки': order.order_number,
            'Город отправления': order.first_point_location.city,
            'Город назначения': order.last_point_location.city,
            'Кластер отправления': order.get_cluster_from(),
            'Кластер назначения': order.get_cluster_to(),
            'Клиент': order.get_customer_name(),
            'Менеджер': order.get_manager_name(),
            'Старт рейса': order.get_start_datetime().strftime('%Y-%m-%d %H:%M:%S'),
            'Цена заявки': order.get_price(),
        })
    df = pd.DataFrame(data)

    # Аналогично формируем DataFrame по заявкам до фильтрации
    data_before = []
    for order in tqdm(orders_before_filtering, desc="Подготовка заявок до фильтрации", unit="заявка"):
        data_before.append({
            'Номер заявки': order.order_number,
            'Город отправления': order.first_point_location.city,
            'Город назначения': order.last_point_location.city,
            'Кластер отправления': order.get_cluster_from(),
            'Кластер назначения': order.get_cluster_to(),
            'Цена заявки': order.get_price(),
        })
    df_before = pd.DataFrame(data_before)

    # Агрегация по кластерам
    df_agg = aggregate_orders(
        df,
        group_cols=['Кластер отправления', 'Кластер назначения'],
        df_before=df_before
    )

    # Агрегация по городам
    df_city_agg = aggregate_orders(
        df,
        group_cols=['Город отправления', 'Город назначения'],
        df_before=df_before
    )

    # Сохраняем в Excel
    output_path = os.path.join("output", filename)
    os.makedirs("output", exist_ok=True)

    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name='OrdersWithClusters')
        df_agg.to_excel(writer, index=False, sheet_name='ClustersAgg')
        df_city_agg.to_excel(writer, index=False, sheet_name='CitiesAgg')
    print(f"Данные кругорейсов успешно сохранены в файл: {output_path}")

In [15]:
# Кластеризуем координаты отправления и назначения
radius = 100.0 # Радиус кластеризации в км
cluster_centroids = cluster_coordinates_from_train_orders(train_orders, radius)


=== Кластеризация координат отправления и назначения ===
Извлекаем координаты...
Найдено 4829 уникальных пар координат

Выполняем кластеризацию с параметрами: radius=100.0km
Всего кластеров: 212


In [16]:
# Сохраняем в excel
save_clustering_results(orders, cluster_centroids, radius)

Результаты кластеризации сохранены в: output/clusters.xlsx
  - Лист 'OrderByCluster': детализация по заявкам
  - Лист 'ClusterDetails': детализация по кластерам


In [17]:
print("\n=== Присваиваем кластеры заявкам ===")
for order in tqdm(orders, desc="Обработка заявок", unit="заявка"):
    from_cluster = find_cluster_for_point(order.get_coordinates_from(), cluster_centroids, radius)
    to_cluster = find_cluster_for_point(order.get_coordinates_to(), cluster_centroids, radius)
    order.set_clusters(from_cluster, to_cluster)


=== Присваиваем кластеры заявкам ===


Обработка заявок: 100%|██████████| 65554/65554 [01:07<00:00, 970.66заявка/s] 


In [18]:
# Убираем экстремумы по цене и заявки, у которых кластер прибытия равен кластеру отправления
orders_before_filtering = [order for order in orders if order.get_cluster_from() != order.get_cluster_to()]
orders = filter_orders_by_cluster_price_median(orders_before_filtering)

train_orders = [order for order in train_orders if order.get_cluster_from() != order.get_cluster_to()]
control_orders = [order for order in control_orders if order.get_cluster_from() != order.get_cluster_to()]
print(f"Заявок с разными кластерами отправления и назначения: {len(orders)}")

Заявок с разными кластерами отправления и назначения: 59780


In [19]:
# Сохраняем в excel детализацию по заявкам и агрегатам
orders_with_clusters_to_xlsx(orders, orders_before_filtering)

Подготовка заявок до фильтрации: 100%|██████████| 63036/63036 [00:00<00:00, 530062.81заявка/s]


Данные кругорейсов успешно сохранены в файл: output\orders_with_clusters.xlsx


In [20]:
# @title Методы для построения графа
def build_location_graph(train_orders, verbose_excel: bool = False, top_customers_count: int = 3):
    """
    Строит граф на основе обучающей выборки заявок

    Args:
        train_orders: Список объектов Order из обучающей выборки
        verbose_excel: Флаг для вывода детализации в файл
        top_customers_count: Количество топовых клиентов для сохранения
    Returns:
        nx.Graph: Граф с локациями в качестве вершин и заявками в качестве ребер
    """
    # Преобразуем заявки в DataFrame
    data = []
    for order in train_orders:
        data.append({
            'source': order.get_cluster_from(),
            'target': order.get_cluster_to(),
            'travel_time': order.get_travel_time(),
            'travel_time_group': order.get_travel_time_group(),
            'price': order.get_price(),
            'customer': order.get_customer_name(),
            'date': order.first_point_date_time_start.date(),
            'weekday': order.first_point_date_time_start.weekday(),
            'distance': order.distance
        })

    df = pd.DataFrame(data)

    # Создаем граф
    G = nx.DiGraph()

    # Группируем данные по ребрам
    grouped = df.groupby(['source', 'target'])

    # Вычисляем периоды
    days_in_period = (df['date'].max() - df['date'].min()).days + 1
    weekday_counts = weekday_counts_by_train_orders(train_orders)

    # Добавляем ребра
    for (source, target), group in grouped:
        # Добавляем вершины
        G.add_node(source)
        G.add_node(target)

        # Группируем по группам времени в пути
        time_groups = group.groupby('travel_time_group')

        # Создаем словари для хранения метрик по группам времени
        weights = {}                    # travel_time_group -> количество заявок
        avg_times = {}                  # travel_time_group -> среднее время в пути (часы)
        avg_prices = {}                 # travel_time_group -> средняя цена заказа (рубли)
        days_frequencies = {}           # travel_time_group -> частота заявок за весь период
        weekday_frequencies = {}        # travel_time_group -> {день_недели -> частота}
        top_customers = {}              # travel_time_group -> строка с топ клиентами и их долями
        median_distances = {}           # travel_time_group -> медианное расстояние (км)

        for travel_time_group, time_group in time_groups:
            weights[travel_time_group] = len(time_group)
            avg_times[travel_time_group] = time_group['travel_time'].mean()
            avg_prices[travel_time_group] = time_group['price'].mean()
            days_frequencies[travel_time_group] = time_group['date'].nunique() / days_in_period
            median_distances[travel_time_group] = time_group['distance'].median()

            # Частота по дням недели
            weekday_freq = {}
            for day in range(7):
                day_count = time_group[time_group['weekday'] == day]['date'].nunique()
                weekday_freq[day] = day_count / weekday_counts.get(day, 1)
            weekday_frequencies[travel_time_group] = weekday_freq

            # Топ клиенты
            if time_group['customer'].notna().any():
                customer_shares = time_group['customer'].value_counts(normalize=True)
                top_customers_list = customer_shares.head(top_customers_count)
                top_customers_str = "; ".join([
                    f"{name} - {share:.2f}"
                    for name, share in top_customers_list.items()
                ])
                top_customers[travel_time_group] = top_customers_str
            else:
                top_customers[travel_time_group] = ""

        # Добавляем ребро с метриками по группам времени
        G.add_edge(source, target,
                   weight=weights,
                   avg_time=avg_times,
                   avg_price=avg_prices,
                   days_frequency=days_frequencies,
                   weekday_frequencies=weekday_frequencies,
                   top_customers=top_customers,
                   median_distance=median_distances)

    # Сохраняем граф в Excel
    if verbose_excel:
        graph_to_xlsx(G, "graph.xlsx")

    return G

def graph_to_xlsx(G, filename):
    """
    Сохраняет граф в Excel файл

    Args:
        G: Граф NetworkX
        filename (str): Имя файла Excel
    """
    # Создаем список для данных
    data = []

    # Добавляем данные по каждому ребру
    for source, target, edge_data in G.edges(data=True):
        for travel_time_group, _ in edge_data['weight'].items():
            row_data = {
                'Город отправления': source,
                'Город назначения': target,
                'Группа времени в пути, по 48 ч.': travel_time_group,
                'Количество заявок': edge_data['weight'][travel_time_group],
                'Среднее время в пути, ч.': round(edge_data['avg_time'][travel_time_group], 2),
                'Средняя цена заказа, руб.': round(edge_data['avg_price'][travel_time_group], 2),
                'Частота заявок за весь период': round(edge_data['days_frequency'][travel_time_group], 4),
                'Частота заявок понедельник': round(edge_data['weekday_frequencies'][travel_time_group][0], 4),
                'Частота заявок вторник': round(edge_data['weekday_frequencies'][travel_time_group][1], 4),
                'Частота заявок среда': round(edge_data['weekday_frequencies'][travel_time_group][2], 4),
                'Частота заявок четверг': round(edge_data['weekday_frequencies'][travel_time_group][3], 4),
                'Частота заявок пятница': round(edge_data['weekday_frequencies'][travel_time_group][4], 4),
                'Частота заявок суббота': round(edge_data['weekday_frequencies'][travel_time_group][5], 4),
                'Частота заявок воскресенье': round(edge_data['weekday_frequencies'][travel_time_group][6], 4),
                'Топовые клиенты': edge_data['top_customers'][travel_time_group]
            }

            data.append(row_data)

    # Создаем DataFrame
    df = pd.DataFrame(data)

    # Сохраняем в Excel
    output_path = os.path.join("output", filename)
    os.makedirs("output", exist_ok=True)

    df.to_excel(output_path, index=False, sheet_name='GraphEdges')
    print(f"Данные графа успешно сохранены в файл: {output_path}")

def print_graph_statistics(G):
    """
    Выводит статистику графа

    Args:
        G: Граф NetworkX
    """
    # Выводим статистику графа
    print(f"\nСтатистика графа:")
    print(f"Количество вершин (локаций): {G.number_of_nodes()}")
    print(f"Количество ребер (маршрутов): {G.number_of_edges()}")

    # Выводим процентили частот заявок по дням недели
    for i in range(7):
        edges = list(G.edges(data='weekday_frequencies'))
        weekday_frequencies = []
        for edge in edges:
            for travel_time_group, weekday_frequency in edge[2].items():
                weekday_frequencies.append(weekday_frequency[i])
        # Считаем долю ребер с частотой >= 20%, 40%, 60%, 80% для каждого дня недели
        thresholds = [0.2, 0.4, 0.6, 0.8]
        total = len(weekday_frequencies)
        if total == 0:
            print(f"Нет данных для дня недели {i}")
            continue
        for threshold in thresholds:
            count = sum(1 for freq in weekday_frequencies if freq >= threshold)
            share = count / total
            print(f"Доля ребер с частотой >= {int(threshold*100)}% для дня недели {i}: {int(share*100)}%")


def weekday_counts_by_train_orders(train_orders):
    """
    Считает количество дней недели за период в обучающей выборке

    Args:
        train_orders: Список объектов Order из обучающей выборки

    Returns:
        dict: Словарь с количеством дней недели за период
    """
    # Находим минимальную и максимальную даты публикации
    min_date = min(order.get_start_datetime().date() for order in train_orders)
    max_date = max(order.get_start_datetime().date() for order in train_orders)

    # Создаем словарь для подсчета дней недели
    weekday_counts = defaultdict(int)
    current_date = min_date
    while current_date <= max_date:
        weekday_counts[current_date.weekday()] += 1
        current_date += timedelta(days=1)
    return weekday_counts

In [21]:
# Строим граф локаций
print("\n=== Построение графа локаций ===")
top_customers_count = 3 # Количество топовых клиентов для детализации
G = build_location_graph(train_orders, top_customers_count)
print_graph_statistics(G)


=== Построение графа локаций ===
Данные графа успешно сохранены в файл: output\graph.xlsx

Статистика графа:
Количество вершин (локаций): 212
Количество ребер (маршрутов): 2051
Доля ребер с частотой >= 20% для дня недели 0: 41%
Доля ребер с частотой >= 40% для дня недели 0: 17%
Доля ребер с частотой >= 60% для дня недели 0: 8%
Доля ребер с частотой >= 80% для дня недели 0: 4%
Доля ребер с частотой >= 20% для дня недели 1: 40%
Доля ребер с частотой >= 40% для дня недели 1: 15%
Доля ребер с частотой >= 60% для дня недели 1: 8%
Доля ребер с частотой >= 80% для дня недели 1: 4%
Доля ребер с частотой >= 20% для дня недели 2: 40%
Доля ребер с частотой >= 40% для дня недели 2: 16%
Доля ребер с частотой >= 60% для дня недели 2: 7%
Доля ребер с частотой >= 80% для дня недели 2: 4%
Доля ребер с частотой >= 20% для дня недели 3: 32%
Доля ребер с частотой >= 40% для дня недели 3: 11%
Доля ребер с частотой >= 60% для дня недели 3: 5%
Доля ребер с частотой >= 80% для дня недели 3: 2%
Доля ребер с ч

In [22]:
# Сохраняем в excel
graph_to_xlsx(G, "graph.xlsx")

Данные графа успешно сохранены в файл: output\graph.xlsx


In [23]:
# @title Методы для построния кругорейсов
class CircularRoute:
    """
    Класс для хранения кругорейса в виде графа
    """
    def __init__(self, order: Order, unloading_hours: float = 6, stopover_hours: float = 8, radius: float = 50):
        self.order = order
        self.unloading_hours = unloading_hours
        self.stopover_hours = stopover_hours
        self.radius = radius
        self.graph = nx.DiGraph()  # Направленный граф для кругорейса

        # Добавляем два первых шага на основе фактических данных заявки
        start_city = order.get_cluster_from()
        end_city = order.get_cluster_to()

        # Рассчитываем время для стартового города (фактические данные заявки)
        start_load_start_time = order.get_start_datetime()
        start_load_end_time = start_load_start_time + timedelta(hours=self.unloading_hours)

        # Рассчитываем время для конечного города (фактические данные заявки)
        end_arrival_time = order.get_end_datetime()
        end_unloading_end_time = end_arrival_time + timedelta(hours=self.unloading_hours)

        # Добавляем города в граф с временем посещения
        self.graph.add_node(start_city,
                           arrival_time=None,  # Отсутствует
                           unloading_end_time=None,  # Отсутствует
                           load_start_time=start_load_start_time,
                           load_end_time=start_load_end_time)

        self.graph.add_node(end_city,
                           arrival_time=end_arrival_time,
                           unloading_end_time=end_unloading_end_time,
                           load_start_time=None,  # Отсутствует
                           load_end_time=None)  # Отсутствует

        # Добавляем ребро между городами с фактическими данными заявки
        self.graph.add_edge(start_city, end_city,
                           step_price=order.get_price(),
                           step_distance=order.distance,
                           step_frequency=1,
                           step_top_customers="")

    def get_last_city(self) -> str:
        """
        Возвращает крайнюю точку (название города) в графе кругорейса.

        Returns:
            str: Название города
        """
        # Возвращаем город с максимальным временем завершения разгрузки (unloading_end_time)
        # Фильтруем узлы, у которых есть unloading_end_time
        nodes_with_unloading_time = [(city, data) for city, data in self.graph.nodes(data=True)
                                    if data.get('unloading_end_time') is not None]

        if not nodes_with_unloading_time:
            # Если нет узлов с unloading_end_time, возвращаем последний добавленный узел
            return list(self.graph.nodes())[-1]

        return max(nodes_with_unloading_time, key=lambda x: x[1]['unloading_end_time'])[0]

    def get_start_city(self) -> str:
        """
        Возвращает стартовую точку кругорейса

        Returns:
            str: Название города
        """
        return self.order.get_cluster_from()

    def is_step_exists(self, from_city: str, to_city: str) -> bool:
        """
        Проверяет, существует ли ребро между двумя городами в графе кругорейса.
        """
        return self.graph.has_edge(from_city, to_city)

    def get_route_frequency(self) -> float:
        """
        Возвращает частоту маршрута кругорейса.
        """
        return math.prod([edge[2]['step_frequency'] for edge in self.graph.edges(data=True)])

    def add_step(self, city: str, edge_data: dict, travel_time_group: str, unloading_hours: float, stopover_hours: float):
        """
        Добавляет шаг в кругорейс и обновляет граф

        Args:
            city: Город назначения
            edge_data: Данные ребра
            travel_time_group: Группа времени в пути
            unloading_hours: Нормативная длительность разгрузки в конечной точке плеча в часах
            stopover_hours: Нормативная длительность стоянки между маршрутами в часах
        """
        current_city = self.get_last_city()
        current_node_data = self.graph.nodes[current_city]

        # Обновляем время загрузки для текущего города
        current_node_data['load_start_time'] = current_node_data['unloading_end_time'] + timedelta(hours=stopover_hours)
        current_node_data['load_end_time'] = current_node_data['load_start_time'] + timedelta(hours=unloading_hours)

        # Рассчитываем время прибытия в новый город
        arrival_time = current_node_data['load_end_time'] + timedelta(hours=edge_data['avg_time'][travel_time_group])

        # Добавляем город в граф с временем посещения
        if self.graph.has_node(city):
            self.graph.nodes[city]['arrival_time'] = arrival_time
            self.graph.nodes[city]['unloading_end_time'] = arrival_time + timedelta(hours=unloading_hours)
        else:
            self.graph.add_node(city,
                            arrival_time=arrival_time,
                            unloading_end_time=arrival_time + timedelta(hours=unloading_hours),
                            load_start_time=None,
                            load_end_time=None)

        # Добавляем ребро с детализацией
        self.graph.add_edge(current_city, city,
                           step_price=edge_data['avg_price'][travel_time_group],
                           step_distance=edge_data['median_distance'][travel_time_group],
                           step_frequency=edge_data['weekday_frequencies'][travel_time_group][current_node_data['load_start_time'].weekday()],
                           step_top_customers=edge_data['top_customers'][travel_time_group])

    def remove_last_step(self):
        """
        Удаляет последний шаг из кругорейса
        """
        last_city = self.get_last_city()
        if last_city == self.get_start_city():
            self.graph.nodes[last_city]['arrival_time'] = None
            self.graph.nodes[last_city]['unloading_end_time'] = None
            # Удаляем последнее добавленное ребро
            edges = list(self.graph.edges)
            if edges:
                last_edge = edges[-1]
                self.graph.remove_edge(*last_edge)
        else:
            if len(self.graph.nodes) > 2:
                # Удаляем последний город
                self.graph.remove_node(last_city)
                prev_last_city = self.get_last_city()
                self.graph.nodes[prev_last_city]['load_start_time'] = None
                self.graph.nodes[prev_last_city]['load_end_time'] = None

    def get_steps_count(self) -> int:
        """
        Возвращает количество шагов в кругорейсе
        """
        return len(self.graph.edges)

    def get_route_sequence(self) -> List[Tuple[str, dict]]:
        """
        Возвращает последовательность городов в кругорейсе
        """
        # Сортируем города по дате прибытия (arrival_time)
        # Фильтруем узлы, у которых есть arrival_time
        nodes_with_arrival_time = [(city, data) for city, data in self.graph.nodes(data=True)
                                  if data.get('arrival_time') is not None]

        # Сортируем узлы с arrival_time
        sorted_nodes = sorted(nodes_with_arrival_time, key=lambda node: node[1]['arrival_time'])

        return sorted_nodes

    def get_route_sequence_by_edges(self) -> List[Tuple[str, str, dict]]:
        """
        Возвращает последовательность ребер в кругорейсе
        """
        # Фильтруем ребра, у которых есть arrival_time в конечной точке
        edges_with_arrival_time = [(from_city, to_city, data) for from_city, to_city, data in self.graph.edges(data=True)
                                  if self.graph.nodes[to_city].get('arrival_time') is not None]

        # Сортируем ребра с arrival_time
        sorted_edges = sorted(edges_with_arrival_time, key=lambda edge: self.graph.nodes[edge[1]]['arrival_time'])

        return sorted_edges

    def get_route_sequence_with_arrival_time(self) -> str:
        """
        Возвращает последовательность городов с датами прибытия
        """
        if not self.graph.nodes:
            return ""

        route_sequence = self.get_route_sequence()
        route_str = []
        for city, node_data in route_sequence:
            arrival_time = node_data.get('arrival_time')
            if arrival_time:
                route_str.append(f"{city} ({arrival_time.strftime('%Y-%m-%d %H:%M:%S')})")
            else:
                route_str.append(city)

        return " -> ".join(route_str)

    def get_total_price(self) -> float:
        """
        Возвращает общую стоимость кругорейса
        """
        return sum([edge_data.get('step_price', 0) for _, _, edge_data in self.graph.edges(data=True)])

    def get_total_distance(self) -> float:
        """
        Возвращает общее расстояние кругорейса
        """
        raw_distance = sum([edge_data.get('step_distance', 0) for _, _, edge_data in self.graph.edges(data=True)])
        additional_distance = (self.radius / 2) * (len(self.graph.edges) - 1)
        return raw_distance + additional_distance

    def get_total_time(self) -> float:
        """
        Возвращает общее время кругорейса в часах
        """
        start_city = self.get_start_city()
        end_city = self.get_last_city()

        start_time = self.graph.nodes[start_city]['load_start_time']
        end_time = self.graph.nodes[end_city]['unloading_end_time']

        if start_time and end_time:
            return (end_time - start_time).total_seconds() / 3600
        return -1

    def is_correct_route(self, control_orders: List[Order]) -> bool:
        """
        Проверяет, были ли все шаги кругорейса в контрольной выборке
        Args:
            control_orders: Список заявок из контрольной выборки
        Returns:
            bool: True, если все шаги кругорейса были в контрольной выборке, False - в противном случае
        """
        route_sequence = self.get_route_sequence()
        correct_route_steps = 0
        for i in range(len(route_sequence)-2):
            from_city = route_sequence[i+1][0]  # Получаем название города
            to_city = route_sequence[i+2][0]    # Получаем название города
            dt = route_sequence[i+1][1].get('load_start_time')  # Получаем данные узла
            if dt and correct_step_example(control_orders, from_city, to_city, dt):
                correct_route_steps += 1
        return correct_route_steps == len(route_sequence)-2

    def get_plan_price_per_day(self) -> float:
      """
      Возвращает плановую выручку в день по кругорейсу
      """
      return self.get_total_price() / self.get_total_time() * 24.0 if self.get_total_time() > 0 else 0

    def get_plan_price_per_km(self) -> float:
        """
        Возвращает плановую выручку по кругорейсу на 1 км
        """
        return self.get_total_price() / self.get_total_distance() if self.get_total_distance() > 0 else 0


def find_circular_routes(G: nx.Graph, order: Order,
                         max_duration_days: int = 5,
                         max_routes: int = 5,
                         stopover_hours: int = 8,
                         unloading_hours: int = 6,
                         min_frequency: float = 0.8,
                         radius: float = 50
                         ) -> CircularRoute:
    """
    Находит все возможные кругорейсы для заданной заявки

    Args:
        G: Граф с локациями и временем в пути
        order: Заявка, для которой ищем кругорейсы
        max_duration_days: Максимальная продолжительность кругорейса в днях
        max_routes: Максимальное количество маршрутов в кругорейсе
        stopover_hours: Нормативная длительность стоянки между маршрутами в часах
        unloading_hours: Нормативная длительность разгрузки в конечной точке плеча в часах
        min_frequency: Минимальная частота маршрута для включения в кругорейс
        radius: Радиус кластеризации в км
    Returns:
        CircularRoute: Объект кругорейса с детализацией
    """
    # Если исходная заявка уже превышает ограничение, возвращаем пустой кругорейс
    if (order.get_travel_time() + unloading_hours > max_duration_days * 24.0) or (order.get_travel_time() is None):
        return []

    all_routes = []

    # Используем DFS для поиска всех путей
    def dfs(circular_route: CircularRoute):
        '''
        circular_route: текущий объект кругорейса
        '''


        # Если вернулись в начальный город, добавляем путь
        current_city = circular_route.get_last_city()
        start_city = circular_route.get_start_city()
        if start_city == current_city:
            all_routes.append(copy.deepcopy(circular_route))
            return

        # Проверяем ограничение на количество плечей до текущей точки
        if circular_route.get_steps_count() >= max_routes:
            return

        current_city_load_start_time = circular_route.graph.nodes[current_city]['unloading_end_time'] + timedelta(hours=stopover_hours)
        current_city_weekday = current_city_load_start_time.weekday()

        # Перебираем всех соседей текущего города
        for neighbor in G.neighbors(current_city):
            # Проверяем отсутствие соседа в графе кругорейса
            if circular_route.graph.has_node(neighbor) and neighbor != start_city:
                continue

            for travel_time_group, _ in G[current_city][neighbor]['weight'].items():
                # Получаем частоту маршрута в заданный день недели
                step_frequency = G[current_city][neighbor]['weekday_frequencies'][travel_time_group][current_city_weekday]

                # Проверяем, что частота маршрута в заданный день недели не менее min_frequency
                if step_frequency < min_frequency or step_frequency * circular_route.get_route_frequency() < min_frequency:
                    continue

                # Получаем время переезда
                edge_time = G[current_city][neighbor]['avg_time'][travel_time_group]

                # Проверяем, не превысит ли добавление этого города ограничение по времени
                total_time = circular_route.get_total_time()
                if total_time + stopover_hours + edge_time + unloading_hours > max_duration_days * 24.0:
                    continue

                # Добавляем город в кругорейс
                edge_data = G[current_city][neighbor]
                circular_route.add_step(neighbor, edge_data, travel_time_group, unloading_hours, stopover_hours)

                # Рекурсивно ищем пути
                dfs(circular_route)

                # Возвращаемся назад
                circular_route.remove_last_step()

    # Создаем объект кругорейса (с двумя первыми шагами)
    circular_route = CircularRoute(order, unloading_hours, stopover_hours, radius)

    # Запускаем DFS
    dfs(circular_route)

    return all_routes



def build_circular_routes(G: nx.DiGraph, control_orders: List[Order], max_duration_days: int = 5,
                          max_routes: int = 5, stopover_hours: int = 8, unloading_hours: int = 6, min_frequency: float = 0.8,
                          radius: float = 50,
                          min_price_per_day: float = 27000,
                          min_price_per_km: float = 55,
                          verbose_excel: bool = False):
    """
    Строит кругорейсы для заявок из контрольной выборки

    Args:
        G: Граф с локациями и временем в пути
        control_orders: Список заявок из контрольной выборки
        max_duration_days: Максимальная продолжительность кругорейса в днях
        max_routes: Максимальное количество маршрутов в кругорейсе
        stopover_hours: Нормативная длительность стоянки между маршрутами в часах
        unloading_hours: Нормативная длительность разгрузки в конечной точке плеча в часах
        min_frequency: Минимальная частота маршрута для включения в кругорейс
        radius: Радиус кластеризации в км
        min_price_per_day: Минимальная плановая выручка в день по кругорейсу, руб.
        min_price_per_km: Минимальная плановая выручка по кругорейсу на 1 км, руб.
        verbose_excel: Флаг для вывода детализации в файл
    Returns:
        Dict[str, List[List[str]]]: Словарь с кругорейсами по номерам заявок
    """

    # Словарь для хранения кругорейсов по заявкам
    circular_routes_by_order = {}

    # Фильтруем заявки, отсутствующие в графе
    control_orders_within_graph_nodes = [order for order in control_orders if G.has_node(order.get_cluster_from()) and G.has_node(order.get_cluster_to())]
    print(f"Контрольных заявок с новыми точками, отсутствующими в графе: {len(control_orders) - len(control_orders_within_graph_nodes)}")
    control_orders_within_graph_edges = [order for order in control_orders_within_graph_nodes
                                         if G.has_edge(order.get_cluster_from(), order.get_cluster_to())]
    print(f"Контрольных заявок с новыми маршрутами, отсутствующими в графе: {len(control_orders_within_graph_nodes) - len(control_orders_within_graph_edges)}")

    orders_for_calculation = control_orders_within_graph_edges

    # Для каждой заявки из контрольной выборки ищем кругорейсы
    for i, order in enumerate(tqdm(orders_for_calculation, desc="Поиск кругорейсов", unit="заявка")):
        # Ищем кругорейсы
        routes = find_circular_routes(G, order, max_duration_days, max_routes, stopover_hours, unloading_hours, min_frequency, radius)
        routes.sort(key=lambda x: x.get_steps_count())

        # Если нашли кругорейсы, сохраняем их
        if routes:
            circular_routes_by_order[order.order_number] = routes

    # Фильтруем кругорейсы по плановой выручке
    circular_routes_by_order = filter_circular_routes_by_revenue(circular_routes_by_order, min_price_per_day, min_price_per_km)

    # Выводим статистику
    total_orders = len(orders_for_calculation)
    orders_with_routes = len(circular_routes_by_order)
    total_routes = sum([len(routes) for routes in circular_routes_by_order.values()])

    print(f"\nСтатистика кругорейсов:")
    print(f"Контрольных заявок: {len(control_orders)}")
    print(f"Заявок для подбора кругорейсов: {total_orders}")
    print(f"Доля покрытия графом контрольной выборки: {total_orders/len(control_orders):.2f}")
    print(f"Заявок с найденными кругорейсами: {orders_with_routes}")
    print(f"Доля заявок с найденными кругорейсами: {orders_with_routes/len(control_orders):.2f}")
    print(f"Всего найдено кругорейсов: {total_routes}")
    print(f"Среднее количество кругорейсов на заявку: {total_routes/total_orders:.2f}")
    print(f"Среднее количество кругорейсов на успешную заявку: {total_routes/orders_with_routes:.2f}")

    # Сохраняем кругорейсы в Excel
    if verbose_excel:
        circular_routes_to_xlsx(G, control_orders, circular_routes_by_order, "circular_routes.xlsx")

    check_accuracy(G, circular_routes_by_order, control_orders)

    return circular_routes_by_order

def circular_routes_to_xlsx(G: nx.DiGraph, control_orders: List[Order], circular_routes_by_order: Dict[str, List[CircularRoute]],
                            filename: str):
    """
    Сохраняет кругорейсы в Excel файл

    Args:
        G: Граф с локациями и временем в пути
        control_orders (List[Order]): Список заявок из контрольной выборки
        circular_routes_by_order (Dict[str, List[CircularRoute]]): Словарь с кругорейсами по номерам заявок
        filename (str): Имя файла Excel
    """
    # Создаем список для данных
    data = []
    steps_data = []

    # Добавляем данные по каждому кругорейсу
    for order_number, routes in tqdm(circular_routes_by_order.items(), desc="Сохранение кругорейсов", unit="заявка"):
        for circular_route in routes:
            order = circular_route.order
            # Добавляем сводные данные

            data.append({
                'Номер заявки': order.order_number,
                'Направление исходной заявки': f"{order.first_point_location.city} -> {order.last_point_location.city}",
                'Клиент исходной заявки': order.get_customer_name(),
                'Менеджер исходной заявки': order.get_manager_name(),
                'Старт рейса, факт': order.get_start_datetime().strftime('%Y-%m-%d %H:%M:%S'),
                'Прибытие на точку разгрузки, факт': order.get_end_datetime().strftime('%Y-%m-%d %H:%M:%S'),
                'Цена заявки, факт, руб.': order.get_price(),
                'Кругорейс, города с датами прибытия': circular_route.get_route_sequence_with_arrival_time(),
                'Количество плечей': circular_route.get_steps_count(),
                'Время в пути, план, дней': circular_route.get_total_time()/24.0,
                'Завершение кругорейса, план': circular_route.graph.nodes[circular_route.get_last_city()]['unloading_end_time'].strftime('%Y-%m-%d %H:%M:%S'),
                'Цена кругорейса, план, руб.': circular_route.get_total_price(),
                'Итоговая частота маршрутов': circular_route.get_route_frequency(),
                'Точность': 1 if circular_route.is_correct_route(control_orders) else 0,
                'Плановая выручка в день по кругорейсу, руб.': circular_route.get_plan_price_per_day(),
                'Плановый выручка по кругорейсу на 1 км, руб.': circular_route.get_plan_price_per_km()
            })

            # Добавляем детализацию по шагам с дополнительными данными
            for i, (from_city, to_city, edge_data) in enumerate(circular_route.get_route_sequence_by_edges()):
                is_first_step = i == 0
                from_node_data = circular_route.graph.nodes[from_city]
                to_node_data = circular_route.graph.nodes[to_city]

                step_example = None if is_first_step else correct_step_example(control_orders, from_city, to_city, from_node_data.get('load_start_time'))

                steps_data.append({
                    'Номер заявки': order.order_number,
                    'Кругорейс': circular_route.get_route_sequence_with_arrival_time(),
                    'Город отправления': from_city,
                    'Город прибытия': to_city,
                    'Фактическое плечо': is_first_step,
                    'Дата старта загрузки в городе отправления': from_node_data.get('load_start_time').strftime('%Y-%m-%d %H:%M:%S') if from_node_data.get('load_start_time') else None,
                    'Дата прибытия в точку разгрузки': to_node_data.get('arrival_time').strftime('%Y-%m-%d %H:%M:%S') if to_node_data.get('arrival_time') else None,
                    'Дата завершения разгрузки в городе прибытия': to_node_data.get('unloading_end_time').strftime('%Y-%m-%d %H:%M:%S') if to_node_data.get('unloading_end_time') else None,
                    'Время в пути, ч.': (to_node_data.get('arrival_time') - from_node_data.get('load_end_time')).total_seconds() / 3600 if to_node_data.get('arrival_time') and from_node_data.get('load_end_time') else None,
                    'Стоимость шага, руб.': edge_data.get('step_price', 0),
                    'Частота маршрута': None if is_first_step else edge_data.get('step_frequency', 0),
                    'Клиенты': order.get_customer_name() if is_first_step else edge_data.get('step_top_customers', ''),
                    'Пример заявки из контрольной выборки': step_example.order_number if step_example else None,
                    'Дата отправления, пример заявки из контрольной выборки': step_example.get_start_datetime().strftime('%Y-%m-%d %H:%M:%S') if step_example else None,
                    'Дата прибытия, пример заявки из контрольной выборки': step_example.get_end_datetime().strftime('%Y-%m-%d %H:%M:%S') if step_example else None,
                    'Цена, пример заявки из контрольной выборки': step_example.get_price() if step_example else None
                })

    # Создаем DataFrame
    df = pd.DataFrame(data)
    steps_df = pd.DataFrame(steps_data)

    # Сохраняем в Excel
    output_path = os.path.join("output", filename)
    os.makedirs("output", exist_ok=True)

    # Сохраняем детализацию по шагам в отдельный лист Excel
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        df.to_excel(writer, index=False, sheet_name='CircularRoutes')
        steps_df.to_excel(writer, index=False, sheet_name='RouteSteps')

    print(f"Данные кругорейсов успешно сохранены в файл: {output_path}")



def check_accuracy(G: nx.Graph, circular_routes_by_order: Dict[str, List[CircularRoute]], control_orders: List[Order]):
    """
    Проверяет точность кругорейсов
    Args:
        G: Граф с локациями и временем в пути
        circular_routes_by_order: Словарь с кругорейсами по номерам заявок [номер заявки, список кругорейсов]
        control_orders: Список заявок из контрольной выборки
    Returns:
        List[Dict[str, float]]: Список метрик для каждого кругорейса [частота, точность]
    """
    routes_metrics = []
    for _, routes in tqdm(circular_routes_by_order.items(), desc="Проверка точности", unit="кругорейс"):
        for circular_route in routes:
            routes_metrics.append({
                'frequency': circular_route.get_route_frequency(),
                'accuracy': 1 if circular_route.is_correct_route(control_orders) else 0
            })

    if not routes_metrics:
        print("Не найдено кругорейсов для проверки точности")
        return routes_metrics

    print(f"Точность: {sum([route['accuracy'] for route in routes_metrics])/len(routes_metrics)*100:.2f}%")

    for i in range(10):
        threshold_left = 0.1 * i
        threshold_right = 0.1 * (i + 1)
        routes = [route for route in routes_metrics if route['frequency'] > threshold_left and route['frequency'] <= threshold_right]
        cnt = len(routes)
        cnt_correct = sum([route['accuracy'] for route in routes])
        if cnt > 0:
            print(f"Для частоты > {threshold_left:.2f} и <= {threshold_right:.2f}: количество - {cnt}, точность - {cnt_correct/cnt*100:.2f}%")

    return routes_metrics

def correct_step_example(control_orders: List[Order], from_city: str, to_city: str, dt_start: datetime) -> Order:
    """
    Пример соответствия шага кругорейса в контрольной выборке
    """
    examples = [order for order in control_orders if
                order.get_cluster_from() == from_city and order.get_cluster_to() == to_city and order.get_start_datetime().date() == dt_start.date()]
    return examples[0] if examples else None

def filter_circular_routes_by_revenue(circular_routes_by_order: Dict[str, List[CircularRoute]],
                                    min_price_per_day: float,
                                    min_price_per_km: float) -> Dict[str, List[CircularRoute]]:
    """
    Фильтрует кругорейсы по плановой выручке

    Args:
        circular_routes_by_order: Словарь с кругорейсами по номерам заявок
        min_price_per_day: Минимальная плановая выручка в день по кругорейсу, руб.
        min_price_per_km: Минимальная плановая выручка по кругорейсу на 1 км, руб.

    Returns:
        Dict[str, List[CircularRoute]]: Отфильтрованный словарь с кругорейсами
    """
    print(f"\nФильтрация кругорейсов по плановой выручке:")
    print(f"Минимальная плановая выручка в день: {min_price_per_day} руб/день")
    print(f"Минимальная плановая выручка на 1 км: {min_price_per_km} руб/км")

    # Сохраняем статистику до фильтрации
    total_routes_before_filtering = sum([len(routes) for routes in circular_routes_by_order.values()])
    orders_with_routes_before_filtering = len(circular_routes_by_order)

    # Фильтруем кругорейсы для каждой заявки
    filtered_circular_routes_by_order = {}
    total_filtered_routes = 0

    for order_number, routes in circular_routes_by_order.items():
        # Фильтруем кругорейсы по критериям выручки
        filtered_routes = [
            route for route in routes
            if route.get_plan_price_per_day() >= min_price_per_day
            and route.get_plan_price_per_km() >= min_price_per_km
        ]

        # Если остались кругорейсы после фильтрации, добавляем заявку в результат
        if filtered_routes:
            filtered_circular_routes_by_order[order_number] = filtered_routes
            total_filtered_routes += len(filtered_routes)

    # Выводим статистику фильтрации
    print(f"Кругорейсов до фильтрации: {total_routes_before_filtering}")
    print(f"Кругорейсов после фильтрации: {total_filtered_routes}")
    print(f"Отфильтровано кругорейсов: {total_routes_before_filtering - total_filtered_routes}")
    print(f"Заявок с кругорейсами до фильтрации: {orders_with_routes_before_filtering}")
    print(f"Заявок с кругорейсами после фильтрации: {len(filtered_circular_routes_by_order)}")
    print(f"Отфильтровано заявок: {orders_with_routes_before_filtering - len(filtered_circular_routes_by_order)}")

    return filtered_circular_routes_by_order

In [24]:
# Параметры подбора кругорейсов
max_duration_days=5  # Максимальная продолжительность кругорейса в днях
max_routes=5         # Максимальное количество плечей в кругорейсе
stopover_hours=8     # Нормативная длительность стоянки между маршрутами в часах
min_frequency=0.4    # Минимальная частота маршрута для включения в кругорейс (больше или равно границе)
min_price_per_day=27000 # Минимальная плановая выручка в день по кругорейсу, руб.
min_price_per_km=55 # Минимальная плановая выручка по кругорейсу на 1 км, руб.

In [16]:
# Строим кругорейсы
print("\n=== Построение кругорейсов ===")

circular_routes = build_circular_routes(
    G,
    control_orders,
    max_duration_days,
    max_routes,
    stopover_hours,
    unloading_hours,
    min_frequency,
    radius,
    min_price_per_day,
    min_price_per_km,
    verbose_excel=True
)


=== Построение кругорейсов ===
Контрольных заявок с новыми точками, отсутствующими в графе: 125
Контрольных заявок с новыми маршрутами, отсутствующими в графе: 2440


Поиск кругорейсов: 100%|██████████| 30532/30532 [00:39<00:00, 764.65заявка/s]



Фильтрация кругорейсов по плановой выручке:
Минимальная плановая выручка в день: 27000 руб/день
Минимальная плановая выручка на 1 км: 55 руб/км
Кругорейсов до фильтрации: 6838
Кругорейсов после фильтрации: 782
Отфильтровано кругорейсов: 6056
Заявок с кругорейсами до фильтрации: 5760
Заявок с кругорейсами после фильтрации: 728
Отфильтровано заявок: 5032

Статистика кругорейсов:
Контрольных заявок: 33097
Заявок для подбора кругорейсов: 30532
Доля покрытия графом контрольной выборки: 0.92
Заявок с найденными кругорейсами: 728
Доля заявок с найденными кругорейсами: 0.02
Всего найдено кругорейсов: 782
Среднее количество кругорейсов на заявку: 0.03
Среднее количество кругорейсов на успешную заявку: 1.07


Сохранение кругорейсов: 100%|██████████| 728/728 [00:13<00:00, 52.59заявка/s]


Данные кругорейсов успешно сохранены в файл: output/circular_routes.xlsx


Проверка точности: 100%|██████████| 728/728 [00:00<00:00, 1355.73кругорейс/s]

Точность: 98.72%
Для частоты > 0.30 и <= 0.40: количество - 162, точность - 96.30%
Для частоты > 0.40 и <= 0.50: количество - 174, точность - 98.85%
Для частоты > 0.50 и <= 0.60: количество - 79, точность - 100.00%
Для частоты > 0.60 и <= 0.70: количество - 1, точность - 0.00%
Для частоты > 0.70 и <= 0.80: количество - 250, точность - 99.60%
Для частоты > 0.90 и <= 1.00: количество - 116, точность - 100.00%


In [25]:
# Список всех кластеров
for node in sorted(G.nodes):
  print(node)

Абакан (кластер 99)
Актау (кластер 189)
Актобе (кластер 168)
Алматы (кластер 104)
Андроново (кластер 204)
Анзорей (кластер 112)
Аркадак (кластер 194)
Артем (кластер 125)
Архангельск (кластер 207)
Архангельск (кластер 92)
Астана (кластер 135)
Астрахань (кластер 12)
Атырау (кластер 200)
Ачинск (кластер 188)
Балаково (кластер 89)
Барановичи (кластер 115)
Барнаул (кластер 0)
Белгород (кластер 129)
Белебей (кластер 77)
Березники (кластер 96)
Бийск (кластер 57)
Бишкек (кластер 127)
Благовещенск (кластер 93)
Борисоглебск (кластер 91)
Бориха (кластер 95)
Братск (кластер 131)
Брянск (кластер 37)
Буденновск (кластер 59)
Бузулук (кластер 163)
Великие Луки (кластер 139)
Великий Новгород (кластер 22)
Вельск (кластер 144)
Владивосток (кластер 176)
Владивосток (кластер 67)
Владимир (кластер 26)
Волгоград (кластер 56)
Волгодонск (кластер 88)
Вологда (кластер 66)
Воронеж (кластер 41)
Выборг (кластер 107)
Вытегра (кластер 208)
Вязьма (кластер 63)
Гай (кластер 157)
Глазов (кластер 80)
Горняк (кластер 117

In [26]:
# Для ручного ввода, список кластеров для ввода в блоке выше
manual_cluster_start = 'Москва (кластер 1)' # Точка исходного кругорейса, где произошел сбой и находится автомобиль
manual_cluster_final = 'Владимир (кластер 48)'  # Конечная точка исходного кругорейса, куда надо вернуться
manual_dt_start = datetime(2025, 8, 5, 10, 0, 0)  # Дата старта кругорейса в формате: год, месяц, день, час, минуты, секунды
manual_max_duration_days = 5  # Плановая длительность движения указанными точками
manual_max_routes = max_routes # Максимальное количество плечей в кругорейсе
manual_min_frequency = min_frequency # Минимальная частота маршрута для включения в кругорейс

In [27]:
# @title Расчет синтетической заявки
for manual_cluster in [manual_cluster_start, manual_cluster_final]:
  if not G.has_node(manual_cluster):
      print(f"Кластер '{manual_cluster}' отсутствует в графе!")

if G.has_node(manual_cluster_start) and G.has_node(manual_cluster_final):
  print("Расчет синтетической заявки")
  synt_order = Order.synthetic(manual_dt_start, manual_dt_start)
  synt_order.set_clusters(manual_cluster_final, manual_cluster_start)
  # Ищем кругорейс
  synt_route = find_circular_routes(G, synt_order, manual_max_duration_days,
                                    manual_max_routes, stopover_hours, unloading_hours, min_frequency=min_frequency)
  # Выводим детализацию найденного кругорейса
  if len(synt_route) > 0 and len(synt_route[0].graph.nodes) > 2:  # Больше чем начальные два города
      print(f"\nНайден кругорейс для синтетической заявки:")
      for idx, circular_route in enumerate(synt_route, 1):
          route_sequence = circular_route.get_route_sequence()
          print(f"\nКругорейс {idx}:")
          for idx_step, (city, node_data) in enumerate(route_sequence[1:], 1):
              arrival_time = node_data.get('arrival_time')
              unloading_end_time = node_data.get('unloading_end_time')
              load_start_time = node_data.get('load_start_time')

              if idx_step == 1:
                  print(f"  Город: {city}, Старт загрузки: {load_start_time}")
              else:
                  print(f"  Город: {city}, Прибытие: {arrival_time}, Завершение разгрузки: {unloading_end_time}, Старт загрузки: {load_start_time}")
          total_time = circular_route.get_total_time()
          total_price = circular_route.get_total_price()
          frequency = circular_route.get_route_frequency()
          print(f"  Общее время кругорейса: {total_time/24:.2f} дн")
          print(f"  Общая стоимость кругорейса: {total_price:.2f}")
          print(f"  Итоговая частота маршрута: {frequency:.2%}")
  else:
      print("Кругорейс для синтетической заявки не найден.")

Кластер 'Москва (кластер 1)' отсутствует в графе!
Кластер 'Владимир (кластер 48)' отсутствует в графе!
